In [112]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

In [53]:
#stop word removal
from nltk.corpus import stopwords
stop={'και','ή', 'γιατί', 'as', 'until ','με','από','από','από','από','από','από','από', 'πριν','μετά','πάνω','κάτω','προς','από','πάνω','κάτω','in','out','on',"πάλι","πάλι","έπειτα","μία","εδώ","εκεί","πότε","πού",'κάθε', 'λίγοι', 'περισσότεροι', 'περισσότεροι', 'άλλοι', 'ορισμένοι', 'μόνο', 'ίδιοι', 'ίδιοι', 'έτσι', 'από ',' πολύ ',' πολύ ',' θα ',' ακριβώς ',' θα πρέπει ','τώρα' } #add related stop word here

In [60]:
df_csv = pd.read_csv('Greek/offenseval-greek-training-v1.csv',sep='\t')
df_test = pd.read_csv('test/testset_taska.tsv',sep='\t')

In [61]:
df_test.head()

,id,tweet
0,2707,@USER Θέλω να των δω από εδώ και εμπρός αν δεν...
1,2251,#survivorgr Α Και 60 φορές και με διαφορετικού...
2,9814,Και μου έλεγε η γυναίκα μου το πρωί πάρε την τ...
3,8949,κατω τα χερια απο τον #κυρανακης
4,6913,"@USER μην μας το παιζεις πονοψυχη,κρυφορατσιστ..."


In [100]:
#marge all data from files that has been read
twee=[]
ids=[]
for i in range(len(df_test)):
    ids.append(df_test['id'][i])
    twee.append(df_test['tweet'][i])


#marge all data from files that has been read
q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(df_csv['subtask_a'][i])
    q_a.append(df_csv['tweet'][i])

print(len(q_a))
print(len(label))
print(len(twee))
print(len(ids))

8743
8743
1544
1544


1544
1544


In [72]:
# remove empty space from the begining of a sentance
TWEETS=[]
for i in twee:
    sen1=i.replace('@USER','')
    sen1=sen1.replace('@user','')
    sen1=sen1.replace("#", ' ')

    sen1=" ".join(sen1.split())
    TWEETS.append(sen1)
twee=TWEETS

In [101]:
# convert labeling to 0 and 1
lb=[]
for i in label:
    if i=='OFF':
        lb.append(1)
    if i=='NOT':
        lb.append(0)
labels=lb
len(labels)

8743

In [81]:
# remove empty space from the begining of a sentance
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=" ".join(sen.split())
    QA.append(sen)
# print(QA)
q_a=QA
len(q_a)

8743

In [102]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=labels

In [105]:
dataframe = comments_attack
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))




In [92]:
# this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [93]:
print(dataframe['label'])

0       1
1       0
2       0
3       0
4       0
       ..
8738    0
8739    0
8740    0
8741    0
8742    0
Name: label, Length: 8743, dtype: int64


In [78]:
comments_attack['label'][990]==0

False

In [106]:
print(dataframe['comment'])

0       @USER Οι μουσουλμάνες που τις βιάζουν έτσι κ α...
1       Η Κάτια προσπαθεί να πείσει οτι δεν είναι ελέφ...
2       Καλά γιατί λες ότι, είσαι νέος αφού γεννήθηκες...
3               Με Φατσεα ξεκινησαμε...... #Kokkinopotami
4                       #gntmgr Κάτια πόσο γλυκιά, εμετός
                              ...                        
8738    Σοβαρό δημοκρατικό έλλειμμα η απουσία γυναικών...
8739                     Γιουροβίζιον βλέπουμε #XFactorGR
8740    Στην Κάτια να δοκιμάσουμε κάτι τέτοιο παρακαλώ...
8741    Γιατι δεν ειναι για μοντέλο! Γι’αυτο κερα Βίκυ...
8742    Κανονικά ο τύπος θα έπρεπε να ντρέπεται μόνο κ...
Name: comment, Length: 8743, dtype: object


In [29]:
# for x in dataframe['comment']:
#     print(x)

In [107]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.04, random_state=42)

In [108]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  8393
test comments length:  350


In [109]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# # ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
# tfidf_vect_ngram.fit(dataframe['comment'])
# xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram_chars.fit(dataframe['comment'])
# xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
# xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)


In [148]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [110]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [25]:
# # Naive Bayes on Word Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
# print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
# print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

In [111]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
# print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, WordLevel TF-IDF:   accuracy: 0.8571428571428571   f1 score: 0.8430848124488849


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [113]:
clf = Pipeline([
    ('vect',tfidf_vect_ngram_chars),
    ('clf', linear_model.LogisticRegression()),
])

clf = clf.fit(train_x, train_y)


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [114]:
Labels=[]
IDE=[]
for i in range(len(twee)):
    IDE.append(ids[i])
    predictions = clf.predict([twee[i]])
    if predictions[0]==1:
        Labels.append('OFF')
    elif predictions[0]==0:
        Labels.append('NOT')
print(len(Labels))
print(len(IDE))

1544
1544


In [115]:
my_dict = { 'q_a' : IDE,'label' : Labels}
df = pd.DataFrame(my_dict)
df.to_csv('submission',index=False)

In [48]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [28]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [108]:
# accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
# print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [130]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [109]:
# accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
# print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [23]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)